In [1]:
import pandas as pd
import re
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
import random
from tqdm import tqdm

# Okt 형태소 분석기 초기화
okt = Okt()

# 불필요한 키워드 리스트 (필터링 대상)
unnecessary_keywords = ['맛있어요', '굿', '최고', '괜찮아요']

foreign_food_keywords = [
    # 이탈리아 음식
    '파스타', '스파게티', '피자', '리조또', '카르보나라', '마르게리타 피자', '페페로니 피자', '볼로네제 파스타', '알프레도 파스타',
    '티라미수', '라자냐', '브루스케타', '카프레제 샐러드', '까르보나라', '프로슈토', '미트볼 파스타', '페스토 파스타', '아란치니', 
    '카넬로니', '뇨끼', '파르페', '젤라토', '아포가토', '판나코타', '크로켓', '크림 파스타',

    # 일본 음식
    '초밥', '스시', '사시미', '덮밥', '라멘', '우동', '소바', '가츠동', '규동', '샤브샤브', '타코야키', 
    '오코노미야키', '텐동', '냉모밀', '회덮밥', '라멘', '야키토리', '돈카츠', '차슈 라멘', '장어덮밥', '스키야키', '규카츠', 
    '미소라멘', '쇼유라멘', '니기리즈시', '우나기',

    # 중국 음식
    '딤섬', '짜장면', '짬뽕', '팔보채', '깐풍기', '양장피', '양꼬치', '북경오리', '마라탕', '마라샹궈', 
    '꿔바로우', '샤오롱바오', '자장면', '군만두', '유린기', '피단두부', '해물짬뽕', '훠궈', '삼겹살볶음', '샤부샤부', '베이징덕',

    # 인도 음식
    '커리', '난', '비리야니', '치킨 티카', '팔락 파니르', '사모사', '라씨', '탄두리 치킨', '치킨 마크니', 
    '차이', '도사', '알루 고비', '파코라', '파니르 티카', '비프 빈달루', '팔락', '차나 마살라', '바터 치킨', '팔락 파니르',

    # 태국 음식
    '팟타이', '똠얌꿍', '카오 팟', '파인애플 볶음밥', '그린 커리', '레드 커리', '쌀국수', '라브', '태국식 치킨윙', 
    '파파야 샐러드', '게팟퐁 커리', '팟시유', '카오만가이', '바질 볶음', '팟타이 새우', '양배추 볶음', '얌운센', '태국식 오믈렛',

    # 베트남 음식
    '쌀국수', '분짜', '짜조', '반미', '반꾸온', '카페쓰어다', '분보후에', '고이꾸온', '반쎄오', '베트남식 스프링롤', 
    '분짜', '반미 샌드위치', '팟라브', '반짠', '베트남 커리', '짜쪼',

    # 멕시코 음식
    '타코', '부리토', '케사디야', '엔칠라다', '나초', '토르티야', '과카몰리', '세비체', '타말레', 
    '알파스토르 타코', '타말레', '퀘사디아', '멕시칸 라이스', '파히타', '타코 볼', '치미창가', '치킨 타코', '비프 타코', '참치 타코',

    # 미국 음식
    '햄버거', '핫도그', '스테이크', '바비큐', '프렌치프라이', '치킨윙', '치즈버거', '클럽 샌드위치', '도넛', '감자튀김', 
    '치킨너겟', '맥앤치즈', '피쉬앤칩스', '치킨 윙', '애플파이', '베이글', '치킨 앤 와플', '치즈 스틱', '프라이드 치킨', 
    '블루베리 팬케이크', '버팔로윙',

    # 프랑스 음식
    '에스카르고', '라따뚜이', '부야베스', '크레페', '크루아상', '바게트', '오믈렛', '타르트', 
    '푸아그라', '카스레', '크림 브륄레', '에끌레어', '무스', '키쉬 로렌', '밀푀유', '프로피테롤', '브리오슈', '타르트 타탱',

    # 지중해 음식
    '파에야', '무사카', '팔라펠', '후무스', '피타 브레드', '바바 가누쉬', '소칼라스', '지중해식 샐러드', '그릭 샐러드', 
    '랍스터 타진', '그리스 요거트', '할루미', '피타브레드', '모로칸 타진', '스파나코피타', '팔라펠',

    # 음료 및 주류
    '카페라떼', '아메리카노', '레드와인', '화이트와인', '샴페인', '모히또', '피나콜라다', '마가리타', 
    '맥주', '에일 맥주', '칵테일', '위스키', '테킬라', '진토닉', '스프라이트', '럼주', '코냑', '사과주', '에스프레소 마티니',

    # 디저트
    '티라미수', '마카롱', '에그타르트', '타르트 타탱', '카스텔라', '치즈케이크', '크로아상', '초콜릿 무스', '팬케이크', 
    '도넛', '와플', '블루베리 머핀', '쿠키', '애플파이', '에끌레어', '초콜릿 트뤼플', '딸기 케이크', '아이스크림 샌드위치', 
    '파르페', '푸딩', '브라우니',
]


# 변별력 있는 키워드를 카테고리별로 분리 (확장된 버전)
distinctive_keywords_category = {
    'taste': [
        '달콤하다', '매콤하다', '짭짤하다', '감칠맛 있다', '담백하다', '상큼하다', '풍미가 깊다', 
        '강렬하다', '고소하다', '새콤달콤하다', '얼큰하다', '깔끔하다', '기름지다', '진하다', 
        '화끈하다', '매운맛이 난다', '단맛이 감돈다', '짠맛이 적당하다', '시큼하다', '스파이시하다',
        '풍부한 맛', '다채로운 맛', '단백하다'
    ],
    'texture': [
        '쫄깃하다', '바삭하다', '부드럽다', '아삭하다', '촉촉하다', '크리미하다', '탱글탱글하다', 
        '쫀득쫀득하다', '겉바속촉이다', '끈적끈적하다', '푸석푸석하다', '사르르 녹는다', '탄력 있다', 
        '쫄깃쫄깃하다', '부드럽게 녹는다', '질감이 독특하다', '질기다', '묵직하다', '가볍다',
        '보드랍다', '바삭바삭하다', '바삭한 식감이 오래간다'
    ],
    'scent': [
        '계피향이 난다', '허브향이 감돈다', '과일 향이 난다', '카카오 향이 진하다', '향긋하다', 
        '달콤한 향이 난다', '고소한 향이 감돈다', '풍미로운 향이 난다', '스모키한 향이 난다', 
        '베리향이 가득하다', '레몬향이 상쾌하다', '초콜릿 향이 짙다', '구운 향이 난다', 
        '매콤한 향이 난다', '화려한 향이 난다', '시원한 향이 난다', '깊고 진한 향이 감돈다', 
        '오랜 여운이 남는 향', '강렬한 향', '부드러운 향이 퍼진다'
    ],
    'service': [
        '서비스가 좋다', '친절하다', '빠르고 신속하다', '체계적인 서비스', '세심한 서비스', 
        '응대가 빠르다', '상냥하다', '정중하다', '신속한 주문 처리', '고객을 배려하다', 
        '프로페셔널하다', '고객 만족도가 높다', '친근하다', '매끄러운 서비스', '맞춤형 서비스', 
        '배려 깊은 서비스', '고객 지향적이다', '세심하게 챙겨준다', '편안한 분위기를 제공하다'
    ],
    'atmosphere': [
        '분위기가 좋다', '아늑하다', '모던하다', '고급스럽다', '활기차다', '로맨틱하다', 
        '차분하다', '세련되다', '프라이빗하다', '힙하다', '트렌디하다', '도심 속 휴식 공간 같다', 
        '편안하다', '자연 친화적이다', '전통적이다', '예술적이다', '빈티지하다', '웅장하다', 
        '레트로 느낌이 난다', '사람이 북적이지 않는다', '자연 채광이 좋다', '아늑한 조명', 
        '감성적이다', '조용하고 차분하다', '라운지 분위기', '음악이 감미롭다', '활기 넘치는 분위기'
    ],
    'recommendation': [
        '재방문 의사가 있다', '지인에게 추천하고 싶다', '다시 오고 싶다', '가족과 함께 오기 좋다', 
        '데이트하기 좋다', '친구들과 오기 좋다', '특별한 날에 오기 좋다', '혼밥하기 좋다', 
        '생일파티 장소로 좋다', '단체로 오기 좋다', '직장 회식 장소로 좋다', '재방문할 것 같다', 
        '아이와 함께 오기 좋다', '행사 장소로 추천한다', '소규모 모임에 좋다', '낮보다는 저녁이 좋다',
        '자주 오고 싶은 곳', '단골이 되고 싶은 곳', '가족끼리 오기에 적합하다', '혼자서도 편하게 올 수 있다'
    ]
}




# 음식 키워드 추출 함수 (가게 메뉴 기반으로 태그 적용)
def extract_food_keywords(reviews, food_menu, num_keywords=10):
    nouns = okt.nouns(reviews)
    noun_freq = pd.Series(nouns).value_counts()
    # 가게 메뉴에 있는 음식만 추출
    food_keywords = [word for word in noun_freq.index if word in food_menu][:num_keywords]
    return food_keywords

# 변별력 키워드 추출 (형용 어구만 추출, 명사 배제)
def extract_distinctive_keywords(reviews, vectorizer, num_keywords=5):
    # TF-IDF 기반으로 변별력 있는 형용 어구만 추출
    tfidf_matrix = vectorizer.transform([reviews])
    keywords = tfidf_matrix.toarray().flatten()
    feature_names = vectorizer.get_feature_names_out()

    # 형용 어구만 추출 (명사 배제)
    top_keywords = [(feature_names[i], keywords[i]) for i in keywords.argsort()[-num_keywords:][::-1] if '형용사' in okt.pos(feature_names[i])]
    
    return [word for word, score in top_keywords]

# 추천 관련 키워드에 가중치 적용 (우선 적용)
def apply_recommendation_weight(keywords, weight=1.5):
    for i, (keyword, score) in enumerate(keywords):
        if keyword in distinctive_keywords_category['recommendation']:
            keywords[i] = (keyword, score * weight)  # 추천 키워드 가중치 적용
    return keywords

# 키워드 분포 조정
def balance_recommendation_keywords(keywords, min_count=1, max_count=3):
    recommendation_keywords = [kw for kw in keywords if kw in distinctive_keywords_category['recommendation']]
    if len(recommendation_keywords) > max_count:
        return keywords[:max_count]  # 너무 많으면 줄이기
    elif len(recommendation_keywords) < min_count:
        # 부족한 경우 추가
        keywords += random.choices(distinctive_keywords_category['recommendation'], k=min_count - len(recommendation_keywords))
    return keywords

# 최종 필터링
def filter_unnecessary_keywords(keywords):
    return [keyword for keyword in keywords if keyword not in unnecessary_keywords]

# 메인 요약 및 키워드 추출 함수
def summarize_with_keywords(review_list, comments, food_menu, num_keywords_food=10, num_keywords_distinctive=5):
    food_keywords_per_store = {}
    distinctive_keywords_per_store = {}

    combined_reviews = [" ".join(comments[res]) for res in review_list]
    vectorizer = TfidfVectorizer().fit(combined_reviews)

    for res in tqdm(review_list, desc="키워드 추출 진행 중"):
        if res in comments:
            review = comments[res]
            pp_review = " ".join(review)
            
            # 음식 키워드 추출
            food_keys = extract_food_keywords(pp_review, food_menu[res], num_keywords_food)
            food_keywords_per_store[res] = food_keys
            
            # 변별력 있는 키워드 추출
            distinctive_keys = extract_distinctive_keywords(pp_review, vectorizer, num_keywords_distinctive)
            distinctive_keys = apply_recommendation_weight(distinctive_keys)  # 추천 키워드 가중치 적용
            distinctive_keys = balance_recommendation_keywords(distinctive_keys)  # 분포 조정
            distinctive_keys = filter_unnecessary_keywords(distinctive_keys)  # 불필요한 키워드 제거
            
            distinctive_keywords_per_store[res] = distinctive_keys

    return food_keywords_per_store, distinctive_keywords_per_store

# 파일 로드 및 데이터 전처리
file_path = './외국음식_0912업데이트.csv'
try:
    data = pd.read_csv(file_path, encoding='utf-8-sig')
except UnicodeDecodeError:
    data = pd.read_csv(file_path, encoding='cp949')

data = data.dropna(subset=['content'])

food_menu = {
    '가게1': ['파스타', '스테이크', '샐러드', '리조또', '카르보나라', '볼로네제 파스타', '크림 파스타', '마르게리타 피자'],
    '가게2': ['초밥', '사시미', '우동', '차슈 라멘', '규동', '가츠동', '스키야키', '회덮밥'],
    '가게3': ['짜장면', '짬뽕', '팔보채', '깐풍기', '양장피', '유산슬', '마라탕', '군만두'],
    '가게4': ['커리', '난', '비리야니', '치킨 티카', '팔락 파니르', '사모사', '탄두리 치킨', '치킨 마크니'],
    '가게5': ['팟타이', '똠얌꿍', '카오 팟', '파인애플 볶음밥', '그린 커리', '쌀국수', '양배추 볶음', '태국식 오믈렛'],
    '가게6': ['타코', '부리토', '케사디야', '엔칠라다', '나초', '토르티야', '과카몰리', '파히타', '치미창가'],
    '가게7': ['햄버거', '핫도그', '스테이크', '프렌치프라이', '치즈버거', '도넛', '맥앤치즈', '피쉬앤칩스'],
    '가게8': ['에스카르고', '라따뚜이', '부야베스', '크레페', '크루아상', '타르트', '푸아그라', '키쉬 로렌']
}


# 리뷰 목록과 코멘트 그룹화
review_list = data['store_name'].unique()
comments = data.groupby('store_name')['content'].apply(list).to_dict()

# 요약 및 키워드 추출 실행
food_keywords_per_store, distinctive_keywords_per_store = summarize_with_keywords(review_list, comments, food_menu)

# 결과 저장
current_date = datetime.now().strftime("%Y%m%d_%H%M%S")
output_file = f'./음식_변별력_키워드_추출_{current_date}.csv'

df = pd.DataFrame({
    'store_name': list(food_keywords_per_store.keys()),
    '음식 키워드': list(food_keywords_per_store.values()),
    '변별력 키워드': list(distinctive_keywords_per_store.values())
})

df.to_csv(output_file, encoding='utf-8-sig', index=False)
print(f"레스토랑별 키워드가 {output_file}에 저장되었습니다.")


키워드 추출 진행 중:   0%|          | 0/692 [00:00<?, ?it/s]


KeyError: '드렁킨타이 목동점'